In [1]:
import torch

In [2]:
x = torch.randn(3, 4, requires_grad=True)
x

tensor([[-0.0794,  0.4353, -0.0415, -0.1946],
        [-0.2159, -0.6213, -0.8655, -0.5107],
        [ 1.0688,  0.2410,  0.2661, -0.3692]], requires_grad=True)

In [22]:
x = torch.randn(3, 4)
x.requires_grad=True
x

tensor([[ 0.6860, -0.7144, -1.4102,  0.6642],
        [ 0.3415,  0.7341, -1.8061,  0.2065],
        [ 1.8626,  0.9246,  0.9433,  0.6166]], requires_grad=True)

In [23]:
b = torch.randn(3, 4, requires_grad=True)
b

tensor([[ 0.3032, -0.5102,  0.6495, -0.0670],
        [ 0.8007,  0.1481, -0.1478,  0.8572],
        [ 0.2000,  0.9142, -0.1171, -1.2335]], requires_grad=True)

In [24]:
t = x * b


In [25]:
y = t.sum()
y

tensor(0.7849, grad_fn=<SumBackward0>)

In [26]:
y.backward()

In [29]:
b.grad

tensor([[ 0.6860, -0.7144, -1.4102,  0.6642],
        [ 0.3415,  0.7341, -1.8061,  0.2065],
        [ 1.8626,  0.9246,  0.9433,  0.6166]])

In [30]:
x.grad

tensor([[ 0.3032, -0.5102,  0.6495, -0.0670],
        [ 0.8007,  0.1481, -0.1478,  0.8572],
        [ 0.2000,  0.9142, -0.1171, -1.2335]])

In [38]:
x = torch.rand(1)
b = torch.rand(1, requires_grad=True)
w = torch.rand(1, requires_grad=True)
y = w * x
z = y + b


In [39]:
z.backward()

In [40]:
w.grad

tensor([0.2743])

In [41]:
import numpy as np
x_values = [i for i in range(11)]
x_train = np.array(x_values, dtype=np.float32)
x_train = x_train.reshape(-1, 1)
x_train.shape

(11, 1)

In [42]:
y_values = [2 * i for i in range(11)]
y_train = np.array(y_values, dtype=np.float32)
y_train = y_train.reshape(-1, 1)
y_train.shape

(11, 1)

In [43]:
import torch
from torch import nn 


In [44]:
class LinearRegressionModel(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LinearRegressionModel, self).__init__()
        self.linear = nn.Linear(input_dim, output_dim)
    
    def forward(self, x):
        out = self.linear(x)
        return out

In [45]:
input_dim = 1
output_dim = 1
model = LinearRegressionModel(input_dim=input_dim, output_dim=output_dim)


In [46]:
model

LinearRegressionModel(
  (linear): Linear(in_features=1, out_features=1, bias=True)
)

In [53]:
epochs = 1000
learning_rate = 1e-2
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
loss_f = nn.MSELoss()


In [56]:
for epoch in range(epochs):
    epoch += 1
    inputs = torch.from_numpy(x_train)
    labels = torch.from_numpy(y_train)
    optimizer.zero_grad()
    outputs = model(inputs)
    loss = loss_f(outputs, labels)
    loss.backward()
    optimizer.step()
    if epoch % 50 == 0:
        print("epoch {}, loss {}".format(epoch, loss))

epoch 50, loss 4.2250738374605135e-07
epoch 100, loss 2.4090866190817906e-07
epoch 150, loss 1.3733014725403336e-07
epoch 200, loss 7.833310888827327e-08
epoch 250, loss 4.482064852595613e-08
epoch 300, loss 2.548042132843875e-08
epoch 350, loss 1.4570074036157621e-08
epoch 400, loss 8.320458100286032e-09
epoch 450, loss 4.7679216130802615e-09
epoch 500, loss 2.7078665976176808e-09
epoch 550, loss 1.5371156569798927e-09
epoch 600, loss 8.879870727618311e-10
epoch 650, loss 5.125212476642105e-10
epoch 700, loss 2.928494946541349e-10
epoch 750, loss 1.718390291172156e-10
epoch 800, loss 1.0206631373410602e-10
epoch 850, loss 5.66009808466994e-11
epoch 900, loss 3.501285372742302e-11
epoch 950, loss 2.214200871264449e-11
epoch 1000, loss 1.2631242406191401e-11


In [57]:
predicted = model(torch.from_numpy(x_train)).data.numpy()
predicted

array([[-6.8403024e-06],
       [ 1.9999942e+00],
       [ 3.9999950e+00],
       [ 5.9999962e+00],
       [ 7.9999971e+00],
       [ 9.9999981e+00],
       [ 1.1999999e+01],
       [ 1.4000000e+01],
       [ 1.6000000e+01],
       [ 1.8000000e+01],
       [ 2.0000002e+01]], dtype=float32)

In [58]:
torch.save(model.state_dict(), "model.pkl")


In [59]:
model.load_state_dict(torch.load("model.pkl"))

<All keys matched successfully>

In [60]:
import torch
from torch import nn
import numpy as np

class LinearRegressionModel(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LinearRegressionModel, self).__init__()
        self.linear = nn.Linear(input_dim, output_dim)
    
    def forward(self, x):
        out = self.linear(x)
        return out

input_dim = 1
output_dim = 1
model = LinearRegressionModel(input_dim, output_dim)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)
loss_f = nn.MSELoss()
learning_rate = 1e-2
epochs = 1000
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)

for epoch in range(epochs):
    inputs = torch.from_numpy(x_train).to(device)
    labels = torch.from_numpy(y_train).to(device)
    optimizer.zero_grad()
    outputs = model(inputs)
    loss = loss_f(outputs, labels)
    loss.backward()
    optimizer.step()
    if epoch % 50 == 0:
        print("epoch {}, loss {}".format(epoch, loss))

epoch 0, loss 40.564308166503906
epoch 50, loss 0.0035180530976504087
epoch 100, loss 0.00200658873654902
epoch 150, loss 0.0011444721603766084
epoch 200, loss 0.0006527711520902812
epoch 250, loss 0.0003723125555552542
epoch 300, loss 0.00021235502208583057
epoch 350, loss 0.00012111983960494399
epoch 400, loss 6.908247451065108e-05
epoch 450, loss 3.940088936360553e-05
epoch 500, loss 2.2472644559456967e-05
epoch 550, loss 1.281810546061024e-05
epoch 600, loss 7.310763521672925e-06
epoch 650, loss 4.169791736785555e-06
epoch 700, loss 2.3786735710018547e-06
epoch 750, loss 1.3566877896664664e-06
epoch 800, loss 7.738690328551456e-07
epoch 850, loss 4.414553131937282e-07
epoch 900, loss 2.5173758899654786e-07
epoch 950, loss 1.4347806143177877e-07
